In [1]:
import pandas as pd
import time

import json
import httplib2
import sys
import codecs
       
import re

import numpy as np
import math
import ast

import warnings
warnings.filterwarnings("ignore")

import json
#!pip install geog
import geog
import shapely.geometry
from shapely.geometry import Point

In [2]:
df = pd.read_csv('new_venues_clean.csv', encoding = "ISO-8859-1") 

In [9]:
### Input
x_0 = 34.05223
y_0 = -118.24368
x_f = 37.77493
y_f = -122.41942

print ("Origin Lat/Long", x_0, y_0 )
print ("Destination Lat/Long", x_f, y_f )
#approximate distance between points (aka "radius")
radius = 80 # approximately 50 miles
print ("Radius", radius )


Origin Lat/Long 34.05223 -118.24368
Destination Lat/Long 37.77493 -122.41942
Radius 80


In [13]:
### Input - Origin Lat/Long, Destination Lat/Long, Radius and Dataframe
def poi_inside_box_optimize(x_0,y_0,x_f,y_f,radius,df):
    #d_t_1 = ((y_f-y_0)**2+(x_f-x_0)**2)**.5
    #print ('Euclidean Distance between Origin and Destination', d_t_1)
    
    #Haversine distance from origin to Destination
    dlat = math.radians(x_f-x_0)
    dlon = math.radians(y_f-y_0)
    haversine_radius = 6371 # km
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(x_f)) \
        * math.cos(math.radians(x_0)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    haversine_dist = haversine_radius * c
    #print ('Haversine Distance between Origin and Destination', d_t)
    
    #actual number of points needed
    no_of_points = round(haversine_dist/radius)-1
    #print ('Number of points', n)
    
    x_delta = (x_f - x_0)/no_of_points
    y_delta = (y_f - y_0)/no_of_points

    #in future iterations this could be points along the route
    list_of_centroids = [(x_0+i*x_delta, y_0+i*y_delta) for i in range(1,no_of_points)]
    #print ('List of Centroids\n', list_of_centroids)
        
    #list of tuples in the form of (x_min, x_max, y_min, y_max)
    polygon_height = radius * 1000  # meters
    polygon_points = 4
    angles = np.linspace(0, 360, polygon_points)
    
    df['in_the_box'] = False
     
    for (x,y) in list_of_centroids:
            #print ("Index of Centroid", centroid_list.index((x,y)))
            p = shapely.geometry.Point([y, x])
            polygon = geog.propagate(p, angles, polygon_height)
            box = shapely.geometry.Polygon(polygon)
            df['in_the_box'] = df.apply(lambda row: Point(row['lng'], row['lat']).within(box), axis = 1)
    return df

In [14]:
%%timeit
df1 = poi_inside_box_optimize(x_0,y_0,x_f,y_f,radius,df)

825 ms ± 55.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
df1[df1['in_the_box'] == True]

,Unnamed: 0,Unnamed: 0.1,name,id,category,address,lat,lng,city,state,price_tier,serves,bucket,big_bucket,like_counts,rating,image_url,in_the_box,centroid_index
117,126,126,IMURA Japanese Restaurant,4bd8f1482e6f0f4776500908,'sushi restaurant',1994C Main St,36.919420,-121.782972,Watsonville,CA,2,"Dinner, Lunch & more",9.0,1,36,8.7,https://fastly.4sqi.net/img/general/300x500/72...,True,99
118,127,127,Erik's DeliCafe,4b54c73bf964a5200acc27e3,'sandwich place',1074 S Green Valley Rd,36.916882,-121.784155,Watsonville,CA,1,Lunch & Dinner,0.0,1,21,8.0,https://fastly.4sqi.net/img/general/300x500/28...,True,99
119,128,128,Freedom Meat Lockers,4bb78c0db35776b0f8c0c701,'bbq joint',160 Hi Grade Ln,36.939099,-121.771735,Freedom,CA,2,"Lunch, Brunch & more",3.0,1,19,8.6,https://fastly.4sqi.net/img/general/300x500/53...,True,99
120,129,129,Cilantros Parrilla Y Cantina,4ba08222f964a5204c7037e3,'mexican restaurant',1934 Main St,36.918582,-121.781350,Watsonville,CA,3,"Happy Hour, Brunch & more",4.0,1,24,7.9,https://fastly.4sqi.net/img/general/300x500/63...,True,99
121,130,130,Cowboy Corner Cafe,4bcb545a0687ef3b6c7cddcc,'breakfast spot',946 Main St,36.916125,-121.766183,Watsonville,CA,1,"Lunch, Brunch & more",0.0,1,20,8.2,https://fastly.4sqi.net/img/general/300x500/Q6...,True,99
122,131,131,Cassidy's Pizza,4bf72dd65efe2d7f069a6834,'pizza place',1400 Freedom Blvd,36.925196,-121.763155,Watsonville,CA,1,NaN,5.0,1,14,8.3,https://fastly.4sqi.net/img/general/300x500/95...,True,99
123,132,132,The Red Apple Cafe,4b1a9dcff964a52082ed23e3,'café',589 Auto Center Dr,36.916688,-121.767088,Watsonville,CA,2,NaN,1.0,1,22,8.1,https://fastly.4sqi.net/img/general/300x500/37...,True,99
124,133,133,Bagel Cafe,4bb6261c6edc76b0c577301c,'bagel shop',na,36.918582,-121.780116,Watsonville,CA,1,NaN,1.0,1,7,7.7,https://fastly.4sqi.net/img/general/300x500/65...,True,99
125,134,134,El Frijolito Restaurant,4babffc5f964a520e4dc3ae3,'mexican restaurant',11 Alexander Street,36.911348,-121.756219,Watsonville,CA,1,"Dinner, Lunch & more",4.0,1,13,8.1,https://fastly.4sqi.net/img/general/300x500/69...,True,99
126,135,135,Sushi Garden,4b33df97f964a520062125e3,'sushi restaurant',1441 Main St,36.916314,-121.775107,Watsonville,CA,2,NaN,9.0,1,12,7.7,https://fastly.4sqi.net/img/general/300x500/39...,True,99


#### Duplicates

In [16]:
df1[df1.duplicated(['name','address'], keep = 'first')]

,Unnamed: 0,Unnamed: 0.1,name,id,category,address,lat,lng,city,state,price_tier,serves,bucket,big_bucket,like_counts,rating,image_url,in_the_box,centroid_index


In [17]:
df1.shape

(4104, 19)

In [ ]:
df1.drop_duplicates(['name','address'], keep = 'first', inplace = True)

In [ ]:
df1.shape

In [18]:
df1.columns.values

array(['Unnamed: 0', 'Unnamed: 0.1', 'name', 'id', 'category', 'address',
       'lat', 'lng', 'city', 'state', 'price_tier', 'serves', 'bucket',
       'big_bucket', 'like_counts', 'rating', 'image_url', 'in_the_box',
       'centroid_index'], dtype=object)

In [19]:
df1.drop(['centroid_index'],axis =1, inplace=True)

In [20]:
df1.to_csv('new_venues_clean.csv')